In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI

c:\code\AI Agent Projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
GEMINI_API_KEY = os.getenv('GEMINI')

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3,google_api_key=GEMINI_API_KEY)